In [15]:
#Download the model
#https://ollama.com/library/llama3.1

In [12]:
!pip install langchain
!pip install langchain-community
!pip install chromadb
!pip install langchain-chroma
!pip install pypdf
!pip install pytest
!pip install ollama

  Using cached build-1.2.2-py3-none-any.whl.metadata (6.2 kB)
  Using cached fastapi-0.114.2-py3-none-any.whl.metadata (27 kB)
  Using cached uvicorn-0.30.6-py3-none-any.whl.metadata (6.6 kB)
  Using cached posthog-3.6.6-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached onnxruntime-1.19.2-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (4.5 kB)
  Using cached opentelemetry_api-1.27.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.27.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached opentelemetry_instrumentation_fastapi-0.48b0-py3-none-any.whl.metadata (2.1 kB)
  Using cached opentelemetry_sdk-1.27.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached tokenizers-0.20.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached PyPika-0.48.9-py2.py3-none-any.whl
  Using cached importlib_resources-6.4.5-py3-none-any.whl.metadata (4.0 kB)
  Using cached grpcio-1.66.1-cp312-cp312-manylinux_

In [1]:
import argparse
import os
import shutil
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain.schema.document import Document
from langchain.vectorstores.chroma import Chroma
import time
from sklearn.decomposition import PCA
import numpy as np
#from langchain_chroma import Chroma

In [2]:
# Load all the PDF documents from the "documents" folder
document_loader = PyPDFDirectoryLoader("documents")
documents = document_loader.load()
print(documents[1])

entry 27 in Xref table invalid but object found


page_content='Neuron
624
Figure 1. Cre-Mediated Recombination and
Cortical Ablation of NT-3
The pattern of Cre-mediated recombination
inFoxg1-Cre mice in the P0 (A) and adult (B)
brains was revealed by crossing animals toaROSA-26 reporter mouse. /H9252-galactosidase
is expressed only after Cre-mediated recom-bination. (C) Southern blot and (D) RNase pro-tection assay of different regions of adultbrain with indicated genotypes show thatthe NT-3flox allele is completely deleted,
and NT-3 mRNA is totally absent in cortex
and hippocampus. Southern blot (C) oftwo NT3
flox;Foxg1-Cre mice (labeled flox/
flox;Foxg1-Cre [1] and [2]) showing completedeletion of the NT-3
floxallele in cortical DNA.
Cre activity is also present in the cerebellumofNT3
flox/flox;Foxg1-Cre mice.
adulthood in the rodent cerebral cortex, hippocampus, mice to a ROSA-26 /H9252-galactosidase reporter mouse
strain (Soriano, 1999). X-gal staining of resulting P0 and and cerebellum (Maisonpierre et al., 1990; Zhou and
Rush, 

In [2]:
# Divides documents into chunks of size 1800 characters with 250 character overlap
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1800,
    chunk_overlap=250,
    length_function=len,
    is_separator_regex=False,
)
# Split the documents into smaller chunks for better processing
chunks = text_splitter.split_documents(documents)
print(chunks[1])

NameError: name 'documents' is not defined

In [3]:
# Initialize the Ollama LLM embedding model with llama3 for generating vector embeddings
ollama_emb = OllamaEmbeddings(model="llama3.1:latest")

In [4]:
# Initialize ChromaDB for storing and retrieving embeddings
CHROMA_PATH = "chroma" # Directory where Chroma will persist the embeddings
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=ollama_emb)

/tmp/ipykernel_13114/1297192462.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  db = Chroma(persist_directory=CHROMA_PATH, embedding_function=ollama_emb)


In [6]:
# (Optional) PCA could be applied to reduce embedding dimensionality
#def apply_pca(embeddings, n_components=50):
#    pca = PCA(n_components=n_components)
#    reduced_embeddings = pca.fit_transform(embeddings)
#    return reduced_embeddings

# Variables to track last page ID and chunk index for metadata
last_page_id = None
current_chunk_index = 0
chunk_texts = []  # Store the text of chunks
chunk_metadata = []  # Store metadata to later add to ChromaDB

# Iterate through each chunk to generate metadata (like source, page, and ID)
for chunk in chunks:
    source = chunk.metadata.get("source") # Document source (file path)
    page = chunk.metadata.get("page") # Page number in the document
    current_page_id = f"{source}:{page}" # Create a unique page ID

    # If the page is the same as the last one, increment the chunk index (for multiple chunks from one page)
    if current_page_id == last_page_id:
        current_chunk_index += 1
    else:
        current_chunk_index = 0 # Reset index if it's a new page

    # Create a unique chunk ID using page ID and chunk index
    chunk_id = f"{current_page_id}:{current_chunk_index}"
    last_page_id = current_page_id # Update last seen page ID

    # Add metadata (chunk ID) to the chunk and store for later
    chunk.metadata["id"] = chunk_id
    chunk_metadata.append(chunk.metadata)
    chunk_texts.append(chunk.page_content)

In [7]:
# Define the batch size for embedding chunks in smaller groups to optimize performance
batch_size = 1000  # This can be adjusted based on system resources

# Start measuring time for embedding process
start_time = time.time()

# Helper function to split chunk_texts into batches
def create_batches(chunk_texts, batch_size):
    for i in range(0, len(chunk_texts), batch_size):
        yield chunk_texts[i:i + batch_size] # Yield batches of size batch_size

# Initialize list to store all embeddings
all_embeddings = []

# Process each batch of text chunks to create embeddings
for batch in create_batches(chunk_texts, batch_size):
    # Use the Ollama embedding model to embed the current batch of text
    batch_embeddings = ollama_emb.embed_documents(batch)
    all_embeddings.extend(batch_embeddings) # Add the embeddings to the list

# End time measurement for embedding process
end_time = time.time()

# Calculate and print the time taken to embed all chunks
time_taken = end_time - start_time
print(f"⏱️ Tiempo tomado para embeber los chunks en lotes: {time_taken:.2f} segundos")


⏱️ Tiempo tomado para embeber los chunks en lotes: 412.84 segundos


In [8]:
# Start time measurement
#start_time = time.time()

# Convert the list of embeddings into a numpy array for easier manipulation
embeddings = np.array(all_embeddings)

# (Optional) 
#reduced_embeddings = apply_pca(embeddings, n_components=50)

# End time measurement
#end_time = time.time()

# Calculate and print the time taken
#time_taken = end_time - start_time
#print(f"⏱️ Tiempo tomado para aplicar PCA a los embeddings: {time_taken:.2f} seconds")

In [9]:
# Start measuring time for adding the embeddings to ChromaDB
start_time = time.time()

# Get all existing items in ChromaDB to avoid duplicating data
existing_items = db.get(include=[])  # IDs are always included by default
existing_ids = set(existing_items["ids"]) # Get the IDs of the existing documents
print(f"Number of existing documents in DB: {len(existing_ids)}")

# Filter out chunks that already exist in the DB by their unique IDs
new_chunks = [chunk for chunk in chunks if chunk.metadata["id"] not in existing_ids]
new_embeddings = [embeddings[i] for i, chunk in enumerate(chunks) if chunk.metadata["id"] not in existing_ids]

# Define a constant batch size for adding documents to ChromaDB
BATCH_SIZE = 1000

# Try adding new documents and embeddings to ChromaDB in batches
try:
    if len(new_chunks):
        print(f"Adding new documents: {len(new_chunks)}")
        for start in range(0, len(new_chunks), BATCH_SIZE):
            print(f"Adding chunk batch {start/BATCH_SIZE}")
            batch = new_chunks[start:start + BATCH_SIZE] # Get a batch of chunks
            batch_embeddings = new_embeddings[start:start + BATCH_SIZE] # Get corresponding embeddings
            batch_ids = [chunk.metadata["id"] for chunk in batch] # Get chunk IDs for the batch
            # Add documents and embeddings to ChromaDB
            db.add_documents(batch, embeddings=batch_embeddings, ids=batch_ids)
        print("New documents added successfully")
    else:
        print("No new documents to add")
except Exception as e:
    print(f"Error occurred during ingestion: {e}")

# End time measurement
end_time = time.time()

# Calculate and print the time taken
time_taken = end_time - start_time
print(f"⏱️ Tiempo tomado para ingestar los embeddings a ChromaDB: {time_taken:.2f} seconds")

Number of existing documents in DB: 0
👉 Adding new documents: 1896
Adding chunk batch 0.0
Adding chunk batch 1.0
✅ New documents added successfully
⏱️ Tiempo tomado para ingestar los embeddings a ChromaDB: 416.98 seconds


In [6]:
# Define a prompt template to structure the question and context for LLM
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms.ollama import Ollama

PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""
# Define the question for the model
#question="Can you give me some common issues from all diseases you know about?"
#question="How can you relate X chromosome and Fabry's disease?"
#question = "Can you tell me the gens involved in Fabry's disease?"
#uestion = "Which disease is related with PLP1?"
question="Can you relate Fabry's disease with Pelizaeus Merzbacher disease?"
#question="Do you know which chromosome is related with Fabry's disease?"
#question="How can you relate Fabry disease with any other disease you know about?"

# Perform similarity search on ChromaDB using the question to find relevant chunks
results = db.similarity_search_with_score(question, k=9)

# Compile the search results into a context for the LLM
context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])

# Prepare the prompt with context and question
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=question)

# Invoke the LLM model to generate an answer based on the context
model = Ollama(model="llama3.1:latest")
response_text = model.invoke(prompt)

# Extract the sources (document IDs) from the search results
sources = [doc.metadata.get("id", None) for doc, _score in results]

# Format the final response with the LLM answer and source documents
formatted_response = f"Response: {response_text}\nSources: {sources}"

# Print the final response along with the sources
print(formatted_response)

Response: Based on the provided references, I can attempt to establish a connection between Fabry's disease and Pelizaeus-Merzbacher disease.

Fabry's disease is mentioned in reference 1 (Ohno S. Sex Chromosomes and Sex-linked Genes. Springer; Berlin: 1967), where it is classified as an X-linked recessive genetic disorder caused by mutations in the GLA gene on the X chromosome. The same book also discusses Pelizaeus-Merzbacher disease, another X-linked condition affecting the central nervous system.

However, a direct relationship between Fabry's disease and Pelizaeus-Merzbacher disease is not explicitly stated in any of the references provided.
Sources: ['documents/jcm-07-00342.pdf:9:2', 'documents/Medicine.pdf:12:0', 'documents/biomolecules-11-00271.pdf:15:3', 'documents/biomolecules-11-00271.pdf:18:4', 'documents/PLP1 proteolipid protein 1 [Homo sapiens (human)] - Gene - NCBI.pdf:2:0', 'documents/biomolecules-11-00271.pdf:15:1', 'documents/GLA galactosidase alpha [Homo sapiens (huma